In [1]:
#imports
import logging
# set seed
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
import utils as ut
import experiment as exp
from evaluation import *
from sklearn.metrics import mean_squared_error
from torch.utils.tensorboard import SummaryWriter
import torch
import random
#define fixed_hyperparams and create a config gen
from configurations import RandomConfigGen, Configuration
from torch import nn
from deep_net import RandomNet
from experiment import run_experiment
import regex as re
from pathlib import *
from plot import *
from sk_models import setup_pls_models_exh, StandardScaler, PLSRegression, LinearRidge

seed = 1
torch.manual_seed(seed)
random.seed(seed + 1)
np.random.seed(seed + 2)
random_state = np.random.RandomState(seed)

In [2]:
#setup wd

file_name = "PLN7.csv"

id_cols =["db_id", "sample_id"]#["db_id", "sample_id"]#["sample_id"]

data_path = Path('D:/workspace/lazydeep/data/soil_data/')
log_path = Path("D:/workspace/lazydeep/experiments/test")
n_components = 10

data_file = data_path / file_name
log_dir = log_path / re.sub(r'\.(?=csv$)[^.]+$', '',file_name)

if not log_dir.exists():
    log_dir.mkdir()
print(log_dir)


D:\workspace\lazydeep\experiments\test\PLN7


In [3]:
data = pd.read_csv(data_file)
nrow, ncol = data.shape
data = ut.sample_data(data,random_state)

n_features = ncol - 1-len(id_cols)
dataset = TabularDataset(data,id_cols = id_cols, cat_cols=None, output_cols=None, ignore_cols= None)
print(data.shape)

(10000, 129)


In [4]:
tt_splitter= train_test_split

train_split1,test_split = tt_splitter([i for i in range(0,len(data))],train_size=5/6,random_state=random_state,shuffle=False)
train_split,val_split = tt_splitter(train_split1,train_size=4/5,random_state=random_state,shuffle=False)
train_data, val_data, test_data = dataset.split(train_split, val_split, test_split,preprocessing=None)

In [5]:
pls = PLSRegression(n_components=n_components)
for k,v in pls.model.__dict__.items():
    print(k)

n_components
deflation_mode
mode
norm_y_weights
scale
algorithm
max_iter
tol
copy
x_rotations_
y_rotations_
x_mean_
x_std_
y_mean_
y_std_
x_scores_
y_scores_
x_weights_
y_weights_
x_loadings_
y_loadings_
coef_
n_features_in_
n_iter_


In [6]:
train_X,train_y = train_data[:]
pls.fit(train_X,train_y)
for k,v in pls.model.__dict__.items():
    print(k)
test_X,test_y = test_data[:]
preds = pls.predict(test_X)

mse = mean_squared_error(test_y,preds)
r2 = r2_score(test_y,preds)
print(f"mse: {mse}, R2: {r2}")

n_components
deflation_mode
mode
norm_y_weights
scale
algorithm
max_iter
tol
copy
x_rotations_
y_rotations_
x_mean_
x_std_
y_mean_
y_std_
x_scores_
y_scores_
x_weights_
y_weights_
x_loadings_
y_loadings_
coef_
n_features_in_
n_iter_
mse: 223.78737323817347, R2: 0.5343077918319914


In [7]:
lr = LinearRidge()

train_X_t = pls.transform(train_X)
test_X_t = pls.transform(test_X)

lr.fit(train_X_t,train_y)

preds = lr.predict(test_X_t)

mse = mean_squared_error(test_y,preds)
r2 = r2_score(test_y,preds)
print(f"mse: {mse}, R2: {r2}")

mse: 223.78737323817353, R2: 0.5343077918319913


In [8]:
uno_x,uno_y = dataset[0]
print("original data")
print(uno_x,uno_y)

trans = pls.transform([uno_x])
print("pls reduced")
print(trans)

original data
[-1.672e-03 -2.850e-04  4.250e-04  8.460e-04  1.156e-03  1.103e-03
  2.790e-04 -8.140e-04 -1.533e-03 -1.751e-03 -1.855e-03 -1.348e-03
 -7.030e-04 -6.930e-04 -1.090e-04 -6.600e-04  0.000e+00  1.330e-04
 -1.209e-03 -1.035e-03 -7.510e-04 -6.560e-04  5.200e-05  7.840e-04
 -7.690e-04 -1.383e-03 -1.220e-03 -1.020e-03 -4.960e-04  1.280e-04
  3.800e-04  3.180e-04  2.800e-04  3.400e-04  3.590e-04  3.400e-04
  4.220e-04  6.490e-04  9.170e-04  1.149e-03  1.193e-03  1.148e-03
  1.174e-03  1.176e-03  1.023e-03  5.140e-04 -1.104e-03 -3.799e-03
 -3.766e-03 -1.705e-03 -5.940e-04 -7.700e-05  1.510e-04  2.010e-04
  2.210e-04  2.240e-04  2.200e-04  1.850e-04  1.780e-04  2.260e-04
  2.280e-04  1.340e-04  1.120e-04  2.770e-04  1.600e-04 -2.100e-05
 -1.400e-05  2.700e-05  8.400e-05  8.800e-05  1.020e-04  1.100e-04
  1.100e-04  1.170e-04  1.360e-04  1.640e-04  1.740e-04  1.770e-04
  1.870e-04  1.950e-04  1.790e-04  1.980e-04  2.000e-04  2.050e-04
  2.150e-04  2.110e-04  1.870e-04  2.000e-04  2.

In [9]:
old_state = deepcopy(pls.state())
pprint(old_state)

{'coef_': array([[-1.43899293e+00],
       [-3.14788596e-01],
       [-1.07069984e+00],
       [ 1.75526039e-01],
       [ 1.92691211e+00],
       [-3.07962586e+00],
       [-5.79268391e+00],
       [-5.71636091e+00],
       [-3.12016833e+00],
       [-2.22347097e+00],
       [-2.15059301e+00],
       [-2.56220792e+00],
       [-1.38016810e+00],
       [-5.23466326e-01],
       [-1.78621382e+00],
       [ 1.50025450e+00],
       [-3.57691348e+00],
       [ 1.60666345e+00],
       [ 1.65799645e+00],
       [-2.75111469e+00],
       [ 2.47979219e+00],
       [ 1.27981771e+00],
       [ 9.69620524e-01],
       [-1.70035919e+00],
       [ 2.82927604e-01],
       [ 2.34667435e+00],
       [ 1.26815182e+00],
       [ 4.57051362e-01],
       [-1.16148314e+00],
       [-1.77355259e+00],
       [ 1.34003481e+00],
       [ 1.39325209e+00],
       [ 1.06112978e+00],
       [ 9.03548990e-02],
       [ 1.10967552e+00],
       [ 2.47717275e+00],
       [ 1.64603405e+00],
       [ 8.18041553e-01],
  

In [10]:
#load state and see if we get the same resulk
pls = PLSRegression(n_components=2)
for k,v in pls.model.__dict__.items():
    print(k)


n_components
deflation_mode
mode
norm_y_weights
scale
algorithm
max_iter
tol
copy
x_rotations_
y_rotations_
x_mean_
x_std_
y_mean_
y_std_
x_scores_
y_scores_
x_weights_
y_weights_
x_loadings_
y_loadings_
coef_
n_features_in_
n_iter_


In [11]:
pls = pls.from_state(old_state)

for k,v in pls.model.__dict__.items():
    print(k)

n_features_in_ 126
x_mean_ [-1.58023747e-03 -1.17899940e-04  7.41367687e-04  1.33241989e-03
  1.70568002e-03  1.44982163e-03  4.37198320e-04 -9.57600510e-04
 -1.93489829e-03 -2.28833453e-03 -2.31544764e-03 -1.65189619e-03
 -8.14439994e-04 -6.01034503e-04  2.01608161e-05 -5.03454395e-04
 -3.15177018e-05 -2.11525653e-05 -1.06854290e-03 -9.02277978e-04
 -9.51151515e-04 -1.07350675e-03 -1.40882988e-04  7.42340384e-04
 -5.36599310e-04 -1.15811326e-03 -1.04308491e-03 -8.50909691e-04
 -3.71875938e-04  1.79014851e-04  4.22845785e-04  3.24985899e-04
  2.88363036e-04  3.56047255e-04  3.19230573e-04  1.96196070e-04
  2.72449145e-04  5.96497000e-04  1.01640309e-03  1.34806811e-03
  1.46041689e-03  1.42243819e-03  1.44293009e-03  1.42482913e-03
  1.08574272e-03  2.19913141e-04 -1.75355731e-03 -4.61085764e-03
 -4.40685539e-03 -2.11122937e-03 -7.66652565e-04 -1.28639364e-04
  1.33399940e-04  2.29485899e-04  2.60266127e-04  2.54507351e-04
  2.45801680e-04  2.25933393e-04  2.03201470e-04  2.22367087e-0

In [12]:
print(pls.model.n_components)

2


In [13]:
train_X,train_y = train_data[:]

test_X,test_y = test_data[:]
preds = pls.predict(test_X)

mse = mean_squared_error(test_y,preds)
r2 = r2_score(test_y,preds)
print(f"mse: {mse}, R2: {r2}")

mse: 223.78737323817347, R2: 0.5343077918319914


In [14]:
#our goal is to reproduce this transformation based off data stored within pls  